In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm

In [2]:
data = pd.read_csv("data/train.csv")

In [3]:
training = data[~data['MISSING_DATA']]

# adding date time, modified from given feature engineering notebook
getdate = lambda x : [tuple([i.year, i.month, i.day, i.hour, i.weekday()]) for i in [datetime.fromtimestamp(x["TIMESTAMP"])]][0]
training[["YR", "MON", "DAY", "HR", "WK"]] = training[["TIMESTAMP"]].apply(getdate, axis=1, result_type="expand")
training = training.drop(columns = ['TIMESTAMP'])

# adding trip time, using the given 15 second formula on polyline
training['TRIP_TIME'] = training['POLYLINE'].apply(lambda x : max(x.count(".")/2-1,0)*15)
training = training.drop(columns = ['POLYLINE'])

C:\Users\ishpa\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [21]:
og = training

In [20]:
ogs = og.sample(frac = 1)
training, testing = ogs[:ogs.shape[0]*8//10], ogs[ogs.shape[0]*8//10:]

In [266]:
from keras.preprocessing.text import Tokenizer

In [303]:
kttrain = training.drop(columns = ['CALL_TYPE','TAXI_ID','TRIP_ID','ORIGIN_CALL','DAY_TYPE','MISSING_DATA','YR','ORIGIN_STAND','DAY'])
kttrain = kttrain.fillna(0).astype({'TRIP_TIME' : int})
kttrain

,MON,HR,WK,TRIP_TIME
0,6,17,6,330
1,6,17,6,270
2,6,17,6,960
3,6,17,6,630
4,6,17,6,420
...,...,...,...,...
1710665,6,16,0,465
1710666,6,16,0,435
1710667,1,2,4,0
1710668,6,8,0,915


In [304]:
tokens = kttrain.astype(str).apply(' '.join, axis=1)
tokens

0          6 17 6 330
1          6 17 6 270
2          6 17 6 960
3          6 17 6 630
4          6 17 6 420
              ...    
1710665    6 16 0 465
1710666    6 16 0 435
1710667       1 2 4 0
1710668     6 8 0 915
1710669    6 12 0 390
Length: 1710660, dtype: object

In [305]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokens)

In [306]:
tokenized = pd.DataFrame()
for col in kttrain.columns:
    tokenized[col] = kttrain[col].apply(lambda x: tokenizer.texts_to_sequences([str(x)])[0])
    
for col in tokenized:
    tokenized[col] = tokenized[col].apply(lambda x: x[0])

In [307]:
tokenized

,MON,HR,WK,TRIP_TIME
0,6,21,6,47
1,6,21,6,60
2,6,21,6,74
3,6,21,6,43
4,6,21,6,35
...,...,...,...,...
1710665,6,22,7,28
1710666,6,22,7,34
1710667,5,4,1,7
1710668,6,11,7,70


In [308]:
X = np.array(tokenized.iloc[:, :-1].values)
y = np.array(tokenized.iloc[:, -1].values)

In [309]:
X

array([[ 6, 21,  6],
       [ 6, 21,  6],
       [ 6, 21,  6],
       ...,
       [ 5,  4,  1],
       [ 6, 11,  7],
       [ 6, 12,  7]], dtype=int64)

In [310]:
y

array([47, 60, 74, ...,  7, 70, 38], dtype=int64)

In [311]:
kttest = testing.drop(columns = ['CALL_TYPE','TAXI_ID','TRIP_ID','ORIGIN_CALL','DAY_TYPE','MISSING_DATA','YR','ORIGIN_STAND','DAY'])
kttest = kttest.fillna(0).astype({'TRIP_TIME' : int})
kttest

,MON,HR,WK,TRIP_TIME
0,8,10,3,1005
1,8,10,3,750
2,8,10,3,840
3,8,10,3,1170
4,8,10,3,2820
...,...,...,...,...
315,12,6,6,885
316,12,6,6,2970
317,12,6,6,3660
318,12,6,6,1200


In [312]:
testokenized = pd.DataFrame()
for col in kttest.columns:
    testokenized[col] = kttest[col].apply(lambda x: tokenizer.texts_to_sequences([str(x)])[0])
    
for col in testokenized:
    testokenized[col] = testokenized[col].apply(lambda x: x[0])

In [313]:
Xtest = np.array(testokenized.iloc[:, :-1].values)
ytest = np.array(testokenized.iloc[:, -1].values)

## training

In [318]:
input_dim = len(np.unique(tokenized.values.flatten()))
hidden_dim = 256
output_dim = len(np.unique(tokenized.values.flatten()))

# Create the LSTM model
model = keras.Sequential()
model.add(keras.layers.Embedding(input_dim=input_dim, output_dim=hidden_dim, input_length=X.shape[1]))
model.add(keras.layers.LSTM(hidden_dim))
model.add(keras.layers.Dense(output_dim, activation='softmax'))

model.compile(loss='crossentropy', optimizer='adam')

# Train the model
model.fit(X, y, batch_size=512, epochs=3)

Epoch 1/3


ValueError: in user code:

    C:\Users\ishpa\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    C:\Users\ishpa\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\ishpa\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\ishpa\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\ishpa\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)

In [ ]:
premaxpreds = model.predict(Xtest)
preds = np.argmax(premaxpreds,axis=1)
print(preds)

ans = tokenizer.sequences_to_texts(np.expand_dims(preds, axis=1))
sqrd= np.sum((testing['TRIP_TIME']-np.array(ans).astype(int))**2/testing.shape[0])
np.sqrt(sqrd)